In [27]:
!pip install transformers

from urllib import request
from bs4 import BeautifulSoup as bs
import re
import nltk
import heapq
from transformers import pipeline

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
url = 'https://en.wikipedia.org/wiki/Toilet'
TotalContent = ""

htmlDoc = request.urlopen(url)

soupObj = bs(htmlDoc, 'html.parser')
paragraphContents = soupObj.findAll('p')

for content in paragraphContents:

  TotalContent += content.text



total_cleaned_content = re.sub(r'\[[0-9]*\]', ' ', TotalContent)
total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

sentence_tokens = nltk.sent_tokenize(total_cleaned_content)

total_cleaned_content = re.sub(r'[^a-zA-Z]', ' ', total_cleaned_content)
total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

print(sentence_tokens)

[' A toilet[n 1] is a piece of sanitary hardware that collects human urine and feces, and sometimes toilet paper, usually for disposal.', 'Flush toilets use water, while dry or non-flush toilets do not.', 'They can be designed for a sitting position popular in Europe and North America with a toilet seat, with additional considerations for those with disabilities, or for a squatting posture more popular in Asia (see squat toilet).', 'In urban areas, flush toilets are usually connected to a sewer system that leads to septic tanks in isolated areas.', 'The waste is known as blackwater and the combined effluent including other sources is sewage.', 'Dry toilets are connected to a pit, removable container, composting chamber, or other storage and treatment device, including urine diversion with a urine-diverting toilet.', 'The technology used for modern toilets varies.', 'Toilets are commonly made of ceramic (porcelain), concrete, plastic, or wood.', 'Newer toilet technologies include dual f

In [29]:


words_tokens = nltk.word_tokenize(total_cleaned_content)

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}

for word in words_tokens:

  if word not in stopwords:

    if word not in word_frequencies.keys():

      word_frequencies[word] = 1
    else:

      word_frequencies[word] += 1


print(words_tokens)


['A', 'toilet', 'n', 'is', 'a', 'piece', 'of', 'sanitary', 'hardware', 'that', 'collects', 'human', 'urine', 'and', 'feces', 'and', 'sometimes', 'toilet', 'paper', 'usually', 'for', 'disposal', 'Flush', 'toilets', 'use', 'water', 'while', 'dry', 'or', 'non', 'flush', 'toilets', 'do', 'not', 'They', 'can', 'be', 'designed', 'for', 'a', 'sitting', 'position', 'popular', 'in', 'Europe', 'and', 'North', 'America', 'with', 'a', 'toilet', 'seat', 'with', 'additional', 'considerations', 'for', 'those', 'with', 'disabilities', 'or', 'for', 'a', 'squatting', 'posture', 'more', 'popular', 'in', 'Asia', 'see', 'squat', 'toilet', 'In', 'urban', 'areas', 'flush', 'toilets', 'are', 'usually', 'connected', 'to', 'a', 'sewer', 'system', 'that', 'leads', 'to', 'septic', 'tanks', 'in', 'isolated', 'areas', 'The', 'waste', 'is', 'known', 'as', 'blackwater', 'and', 'the', 'combined', 'effluent', 'including', 'other', 'sources', 'is', 'sewage', 'Dry', 'toilets', 'are', 'connected', 'to', 'a', 'pit', 'remov

In [30]:
maximum_frequency = max(word_frequencies.values())

for word in word_frequencies.keys():
  word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

print(word_frequencies)


{'A': 0.16326530612244897, 'toilet': 1.0, 'n': 0.05102040816326531, 'piece': 0.01020408163265306, 'sanitary': 0.02040816326530612, 'hardware': 0.01020408163265306, 'collects': 0.02040816326530612, 'human': 0.08163265306122448, 'urine': 0.1836734693877551, 'feces': 0.05102040816326531, 'sometimes': 0.061224489795918366, 'paper': 0.04081632653061224, 'usually': 0.05102040816326531, 'disposal': 0.02040816326530612, 'Flush': 0.030612244897959183, 'toilets': 0.7857142857142857, 'use': 0.29591836734693877, 'water': 0.5306122448979592, 'dry': 0.10204081632653061, 'non': 0.05102040816326531, 'flush': 0.29591836734693877, 'They': 0.09183673469387756, 'designed': 0.08163265306122448, 'sitting': 0.08163265306122448, 'position': 0.061224489795918366, 'popular': 0.02040816326530612, 'Europe': 0.05102040816326531, 'North': 0.01020408163265306, 'America': 0.030612244897959183, 'seat': 0.07142857142857142, 'additional': 0.01020408163265306, 'considerations': 0.01020408163265306, 'disabilities': 0.0408

In [31]:
sentence_scores = {}

for sentence in sentence_tokens:

  for word in nltk.word_tokenize(sentence.lower()):

    if word in word_frequencies.keys():

      if (len(sentence.split(' '))) < 30:

        if sentence not in sentence_scores.keys():
          sentence_scores[sentence] = word_frequencies[word]
        else:
          sentence_scores[sentence] += word_frequencies[word]


points = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)


In [32]:
total_text = ""

for i in points:

    total_text += i
    

In [33]:
summarizer = pipeline("summarization") # model="t5-small"

summary = summarizer(total_text, max_length=128, min_length=30, do_sample=False)



In [34]:
summary[0]["summary_text"]

' A squat toilet (also called "squatting toilet", "natural position toilet", or by many national names) is a toilet of any technology type . Many types of toilets without a water seal exist . Dry toilets are connected to a pit, removable container, composting chamber, or other storage and treatment device .'

In [35]:
print("Bullet Points: ")

for i in points:
    
   print("\n - " + i)

print("\n Summary: " + str(summary[0]["summary_text"]))

Bullet Points: 

 - A squat toilet (also called "squatting toilet", "natural position toilet", or by many national names) is a toilet of any technology type (i.e.

 - Many types of toilets without a water seal (also called dry toilets or "non-flush toilets") exist.

 - Another option is to have one room for body washing (also called "bathroom") and a separate one for the toilet and handwashing sink (toilet room).

 - (Toto, an abbreviation of Tōyō Tōki (東洋陶器 Oriental Ceramics), is used in Japanese comics to visually indicate toilets or other things that look like toilets; see Toilets in Japan.)

 - Passenger train toilets, aircraft lavatories, bus toilets, and ships with plumbing often use vacuum toilets.

 - A vacuum toilet is a flush toilet that is connected to a vacuum sewer system, and removes waste by suction.

 - Flushing toilets can be plumbed to use greywater (previously used for washing dishes, laundry, and bathing) rather than potable water (drinking water).

 - Dry toilets a